In [17]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


you need to install this package

In [18]:
!pip install --q git+https://github.com/m-bain/whisperx.git

  Preparing metadata (setup.py) ... done


In [19]:
import whisperx
import gc

/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [20]:
PATH="/content/gdrive/MyDrive/gdrive"

In [21]:
device = "cuda"
batch_size = 4
compute_type = "float16"
audio_file = PATH+"/GP_Transco/aud-20240305025406001766-5f28b652ac8760c054204aa095bc31e3-C862.wav"

Yes, we were told to use chatGPT ,however , since we would also like the speaker diarization functionality, we will use different pipeline. (whisper based as well)

In [22]:
model = whisperx.load_model("large-v2", device, compute_type=compute_type)

No language specified, language will be first be detected for each audio file (increases inference time).


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.2.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.2.1+cu121. Bad things might happen unless you revert torch to 1.x.


In [23]:
audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio, batch_size=batch_size)

Detected language: en (1.00) in first 30s of audio...


In [ ]:
print(result["segments"])
print(result)

[{'text': " Hey Fidel, I'm sorry, this one. I'm done with the truck. Okay, perfect, perfect. What time is my clock going to kick in tomorrow? Let me see real quick. One second. For tomorrow it says open window. Let me double check that real quick, Fidel.", 'start': 2.551, 'end': 30.111}, {'text': " I have a 8 a.m. appointment time for the delivery. On Wednesday? On Wednesday, yes, sir. All right. Yeah, yeah. But it's a 24-7 facility, so if you have any trouble on the route, just give me a call so we can fix it, okay? All right. Yeah, I'm not leaving until tomorrow. I'm out of time, so just use PC out here.", 'start': 31.032, 'end': 59.087}, {'text': " No, no, it's fine, it's fine. Are you at HQ, right? Yeah, I'm out of time. No, it's fine, you can sit down at the headquarters. Can I use YARMUV? Oh, yeah, YARMUV, it's okay, it's fine. And you can also take the 15 minutes for the post-trip. It won't count like a violation, so you can take the", 'start': 59.531, 'end': 87.449}, {'text': '

Let's make some speaker diarization.

In [ ]:
hf_token=

In [24]:
model_a, metadata = whisperx.load_align_model(language_code=result["language"],
                                              device=device)

result = whisperx.align(result["segments"], model_a,
                        metadata,
                        audio,
                        device,
                        return_char_alignments=False)
diarize_model=whisperx.DiarizationPipeline(use_auth_token=hf_token,device=device)
diarize_segments=diarize_model(audio,min_speakers=2,max_speakers=2)
final_segmentation=whisperx.assign_word_speakers(diarize_segments,result)

In [12]:
print(final_segmentation['segments'][4])

{'start': 11.938, 'end': 14.86, 'text': 'Let me see real quick.', 'words': [{'word': 'Let', 'start': 11.938, 'end': 12.098, 'score': 0.417, 'speaker': 'SPEAKER_01'}, {'word': 'me', 'start': 12.718, 'end': 12.858, 'score': 0.773, 'speaker': 'SPEAKER_01'}, {'word': 'see', 'start': 12.899, 'end': 13.099, 'score': 0.799, 'speaker': 'SPEAKER_01'}, {'word': 'real', 'start': 13.119, 'end': 13.319, 'score': 0.675, 'speaker': 'SPEAKER_01'}, {'word': 'quick.', 'start': 13.379, 'end': 14.86, 'score': 0.638, 'speaker': 'SPEAKER_00'}], 'speaker': 'SPEAKER_01'}


Some cleaning and organization. we will extract the speech segements and the time stamps.

In [25]:
dialog=[]
time_segments=[]
for item in final_segmentation['segments']:
    try:
      dialog.append({item['speaker']:item['text']})
      time_segments.append((item['start'],item['end']))
    except:
      print(item)
      pass

{'start': 130.201, 'end': 130.341, 'text': 'Bye.', 'words': [{'word': 'Bye.', 'start': 130.201, 'end': 130.341, 'score': 0.499}]}


Now we can start using OpenAI...

In [18]:
#!pip install openai
import os
os.environ['OPENAI_API_KEY'] = ''

In [19]:
#!pip install openai

import openai
import typing as t

def completion_block(conversation: t.List[t.Dict[str,str]],
                     temp: float = 0.7,
                     model: str = "gpt-4",
                     tokens: int = 1024) -> str:

    response = openai.chat.completions.create(model=model,
                                              temperature=temp,
                                              max_tokens=tokens,
                                              messages=conversation)

    return response

There are several tasks in our list where OpenAI can help us .
For each of the tasks we will define appropriate prompt.

In [58]:
def match_prompt(my_case,**kwargs):
  dialog=kwargs['dialog']
  match my_case:
      case 'speaker_identity_prompt':
          num_lines=kwargs['num_lines']
          prompt=f"You are given a conversation between a truck driver and a routing operator. Each dictionary has a dialog phrase spoken by one of them. Based on dialog content and logics , you need to correctly assign the dialog lines to each one of them. Return similar list of dictionaries where each line is assigned correctly to each speaker. The dialog contains {num_lines} lines. the output list of dictionaries should contain the same number of lines.\nDialog: {dialog}"
      case 'dialog_summary':
          prompt=f"Given a dialog between a truck driver and a routing operator , produce summary containing the most essential points. Make a list of bulet points with most important dialog outcomes.\nDialog: {dialog}"
      case "dialog_sentiment":
          prompt=f"Rate the overall sentiment of the dialogue on a scale from 1 to 10,  with 1 indicating a highly negative sentiment and 10 denoting a highly positive sentiment. Return number (or range of number only).\nDialog: {dialog}"
      case "dialog_emotions":
          prompt=f"Extract key emotions in call with their score for each ranging from 1 to 10 with 1 indicating a highly negative sentiment and 10 denoting a highly positive sentiment. Refer to each part of dialog and for each part indicate appropriate emotion and it's score in maximum 2 words. Return dictionary with appropriate part of dialog, emotion and score.\nDialog: {dialog}"
      case "question":
          prompt=f"Based on the dialog below, what is the appointment time for delivery?\nDialog: {dialog}"
  return prompt

In [21]:
def get_openai_response(my_case,**kwargs):
    prompt=match_prompt(my_case,**kwargs)
    print(prompt)
    messages=[{"role": "system", "content": prompt},
              {"role": "user", "content": ""}]
    response=completion_block(messages)
    return response.choices[0].message.content

First, we would like to make some dialog corrections ,
1. we know who are the speakers in the dialog - truck driver and operator , thus, we would like to assign the speakers automaticaly without the need to identify manually who was the first one to speak.
2. There is a chance that the diarization tool made some mistakes, based on dialog logics, we would like openAI to correct this.

In [43]:
corrected_dialog=get_openai_response('speaker_identity_prompt',num_lines=len(dialog),dialog=dialog)

You are given a conversation between a truck driver and a routing operator. Each dictionary has a dialog phrase spoken by one of them. Based on dialog content and logics , you need to correctly assign the dialog lines to each one of them. Return similar list of dictionaries where each line is assigned correctly to each speaker. The dialog contains 46 lines. the output list of dictionaries should contain the same number of lines.
Dialog: [{'SPEAKER_01': " Hey Fidel, I'm sorry, this one."}, {'SPEAKER_00': "I'm done with the truck."}, {'SPEAKER_01': 'Okay, perfect, perfect.'}, {'SPEAKER_00': 'What time is my clock going to kick in tomorrow?'}, {'SPEAKER_01': 'Let me see real quick.'}, {'SPEAKER_01': 'One second.'}, {'SPEAKER_01': 'For tomorrow it says open window.'}, {'SPEAKER_01': 'Let me double check that real quick, Fidel.'}, {'SPEAKER_01': ' I have a 8 a.m.'}, {'SPEAKER_01': 'appointment time for the delivery.'}, {'SPEAKER_01': 'On Wednesday?'}, {'SPEAKER_01': 'On Wednesday, yes, sir.

Due to some randomality in answer generation , making complite, production level pipeline out of this , will require some post-processing function.

In [48]:
import ast
corrected_dialog=ast.literal_eval(corrected_dialog)

In [26]:
corrected_dialog=[{'Truck Driver': " Hey Fidel, I'm sorry, this one."},
{'Routing Operator': "I'm done with the truck."},
{'Truck Driver': 'Okay, perfect, perfect.'},
{'Routing Operator': 'What time is my clock going to kick in tomorrow?'},
{'Truck Driver': 'Let me see real quick.'},
{'Truck Driver': 'One second.'},
{'Truck Driver': 'For tomorrow it says open window.'},
{'Truck Driver': 'Let me double check that real quick, Fidel.'},
{'Truck Driver': ' I have a 8 a.m.'},
{'Truck Driver': 'appointment time for the delivery.'},
{'Truck Driver': 'On Wednesday?'},
{'Truck Driver': 'On Wednesday, yes, sir.'},
{'Truck Driver': 'All right.'},
{'Truck Driver': 'Yeah, yeah.'},
{'Truck Driver': "But it's a 24-7 facility, so if you have any trouble on the route, just give me a call so we can fix it, okay?"},
{'Routing Operator': 'All right.'}, {'Routing Operator': "Yeah, I'm not leaving until tomorrow."},
{'Routing Operator': "I'm out of time, so just use PC out here."},
{'Truck Driver': " No, no, it's fine, it's fine."},
{'Truck Driver': 'Are you at HQ, right?'},
{'Routing Operator': "Yeah, I'm out of time."},
{'Truck Driver': "No, it's fine, you can sit down at the headquarters."},
{'Routing Operator': 'They rule with YARMUV?'},
{'Routing Operator': 'They use YARMUV?'},
{'Truck Driver': "Oh, yeah, YARMUV, it's okay, it's fine."},
{'Truck Driver': 'And you can also take the 15 minutes for the post-trip.'},
{'Truck Driver': "It won't count like a violation, so you can take"},
{'Truck Driver': ' the 15 minutes for the post-trip.'},
{'Routing Operator': 'Oh, I could use it with the YARMUV?'},
{'Truck Driver': 'Yeah, yeah, yeah.'},
{'Truck Driver': 'Once you use the YARMUV and park, you can put yourself on non-duty.'},
{'Truck Driver': 'Post-trip?'},
{'Truck Driver': 'For the post-trip.'},
{'Truck Driver': 'Yes, sir.'},
{'Routing Operator': 'Get out of YARMUV to on-duty?'},
{'Truck Driver': 'Exactly, yeah.'},
{'Truck Driver': ' Just before you end with the yard move, put yourself on duty, make the post-trip, and then sleep or burst off or off duty.'},
{'Truck Driver': 'All right.'}, {'Truck Driver': 'Sleep or burst off.'},
{'Truck Driver': 'Yeah.'},
{'Routing Operator': 'All right then.'},
{'Truck Driver': 'All right.'},
{'Routing Operator': 'Thank you.'},
{'Truck Driver': 'Take care.'},
{'Truck Driver': 'Bye-bye.'},
{'Routing Operator': 'All right.'}]

In [27]:
# Let's check that gpt didn't change any content. ,basic checks
for a,b in zip(dialog,corrected_dialog):
    dialog_item=list(a.values())[0]
    dialog_item=dialog_item.strip()
    corrected_item=list(b.values())[0]
    corrected_item=corrected_item.strip()
    if dialog_item!=corrected_item:
       print("Data doesn't match!")

In [28]:
#Let's merge each speaker's lines and organize the time frames
prev_s=time_segments[0][0]
prev_e=time_segments[0][1]
prev_speaker=list(corrected_dialog[0].keys())[0]
prev_line=list(corrected_dialog[0].values())[0]
organized_dialog=[]
organized_segments=[]
for item,time_segment in zip(corrected_dialog[1:],time_segments[1:]):
    cur_speaker=list(item.keys())[0]
    cur_line=list(item.values())[0]
    cur_s=time_segment[0]
    cur_e=time_segment[1]
    if cur_speaker==prev_speaker:
       prev_line+=cur_line
       prev_e=cur_e
    else:
       organized_dialog.append({prev_speaker:prev_line})
       organized_segments.append((prev_s,prev_e))
       prev_speaker=cur_speaker
       prev_line=cur_line
       prev_s=cur_s
       prev_e=cur_e

Fianl organized dialog.

In [57]:
organized_dialog

[{'Truck Driver': " Hey Fidel, I'm sorry, this one."},
 {'Routing Operator': "I'm done with the truck."},
 {'Truck Driver': 'Okay, perfect, perfect.'},
 {'Routing Operator': 'What time is my clock going to kick in tomorrow?'},
 {'Truck Driver': "Let me see real quick.One second.For tomorrow it says open window.Let me double check that real quick, Fidel. I have a 8 a.m.appointment time for the delivery.On Wednesday?On Wednesday, yes, sir.All right.Yeah, yeah.But it's a 24-7 facility, so if you have any trouble on the route, just give me a call so we can fix it, okay?"},
 {'Routing Operator': "All right.Yeah, I'm not leaving until tomorrow.I'm out of time, so just use PC out here."},
 {'Truck Driver': " No, no, it's fine, it's fine.Are you at HQ, right?"},
 {'Routing Operator': "Yeah, I'm out of time."},
 {'Truck Driver': "No, it's fine, you can sit down at the headquarters."},
 {'Routing Operator': 'They rule with YARMUV?They use YARMUV?'},
 {'Truck Driver': "Oh, yeah, YARMUV, it's okay

Let's extract the dialog summary

In [ ]:
dialog_summary=get_openai_response('dialog_summary',dialog=organized_dialog)

Summary of dialog main points:

In [ ]:
dialog_summary="""
- The truck driver (Fidel) has finished with the truck for the day.
- Fidel inquires about the start time of his work shift the following day.
- The routing operator confirms an 8 AM appointment time for delivery on Wednesday at a 24-7 facility.
- Fidel mentions he will not leave until tomorrow due to being out of time and plans to use PC (personal conveyance) due to this.
- The routing operator confirms it is fine for Fidel to stay at the headquarters due to being out of time.
- They discuss the use of YARMUV, which is allowed for the 15 minutes"""

1 . Presonally I'm not familiar with YARMUV. For making a verified pipeline , we will need to account for situations when the quality of recording is low or area specific terms being used that the speech-to-text model is not able to identify/transcribe correctly .
2. The prompt deifnitelly can be adjusted based on requirements.


Let's extract the overall emotional score of the dialog

In [ ]:
import re
dialog_emotional_score=get_openai_response('dialog_sentiment',dialog=organized_dialog)
print("dialog_emotional_score",dialog_emotional_score)

dialog_emotional_score 7


Again, if we want to make a production level pipeline out of this, we will need to make sure that we return numerical scores only (OpenAI , due to being generative tool, intoduces randomality in answers.)


In [ ]:
key_emotions=get_openai_response('dialog_emotions',dialog=organized_dialog)
print("key_emotions",key_emotions)

key_emotions 
[
  {
    "part": "routing operator: Hey Fidel, I'm sorry, this one.",
    "emotion": "Apology",
    "score": 4
  },
  {
    "part": "truck driver: I'm done with the truck.",
    "emotion": "Relief",
    "score": 7
  },
  {
    "part": "routing operator: Okay, perfect, perfect.",
    "emotion": "Satisfaction",
    "score": 8
  },
  {
    "part": "truck driver: What time is my clock going to kick in tomorrow?",
    "emotion": "Curiosity",
    "score": 5
  },
  {
    "part": "routing operator: Let me see real quick.One second.For tomorrow it says open window...",
    "emotion": "Helpfulness",
    "score": 7
  },
  {
    "part": "truck driver: All right.Yeah, I'm not leaving until tomorrow...",
    "emotion": "Acknowledgement",
    "score": 6
  },
  {
    "part": "routing operator: No, no, it's fine, it's fine.Are you at HQ, right?",
    "emotion": "Reassurance",
    "score": 8
  },
  {
    "part": "truck driver: Yeah, I'm out of time.",
    "emotion": "Stress",
    "score":

To make things less expensive, what can be done in the future, is based on the data collected with OpenAI usage, we can train separate model for each functionality/tasks from the described above.

Here we used OpenAI as an emotion classifier. Let's try other libraries and see if there are additional insights that we can extract.
**Experiment**

In [61]:
#!pip install nrclex
from nrclex import NRCLex

In [62]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [82]:
def check_neutrality(items_list):
    all_scores=[item[1] for item in items_list]
    if all(i == all_scores[0] for i in all_scores) and (all_scores[0]<0.2):
       return "neutral",items_list
    return "",items_list

emotions=[]
scores_=[]
for item in organized_dialog:

    dialog_line=list(item.values())[0]
    em=NRCLex(dialog_line)
    all_emotions=em.top_emotions
    is_neutral,defined_scores=check_neutrality(all_emotions)
    scores_.append(defined_scores)

    if len(is_neutral)==0:
      emotions.append({list(item.keys())[0]:','.join([item[0] for item in all_emotions])})
    else:
      emotions.append({list(item.keys())[0]:is_neutral})

here are the emotions detected by other library.

In [90]:
print(scores_[:2])

[[('fear', 0.0), ('anger', 0.0), ('anticip', 0.0), ('trust', 0.0), ('surprise', 0.0), ('positive', 0.0), ('negative', 0.0), ('sadness', 0.0), ('disgust', 0.0), ('joy', 0.0)], [('trust', 1.0)]]


In [88]:
emotions

[{'Truck Driver': 'neutral'},
 {'Routing Operator': 'trust'},
 {'Truck Driver': 'trust,positive,joy,anticipation'},
 {'Routing Operator': 'anticipation'},
 {'Truck Driver': 'trust,positive,anticipation'},
 {'Routing Operator': 'anticipation'},
 {'Truck Driver': 'neutral'},
 {'Routing Operator': 'anticipation'},
 {'Truck Driver': 'neutral'},
 {'Routing Operator': 'fear,trust'},
 {'Truck Driver': 'neutral'},
 {'Routing Operator': 'neutral'},
 {'Truck Driver': 'trust,positive'},
 {'Routing Operator': 'neutral'},
 {'Truck Driver': 'neutral'},
 {'Routing Operator': 'neutral'},
 {'Truck Driver': 'neutral'},
 {'Routing Operator': 'neutral'},
 {'Truck Driver': 'neutral'}]

 - For entries where all the scores of the listed emotions where equal and lower than 0.2 , we assigned neutral. (in such entries a long list with sometimes controversial emotions is present ,seems that the model strugled wiht the identification.)
 - On the general ,the package is not so suitable for percise emotion identification .
 - we can make general summary based on the emotion identified for each phrase that the leading emotional level is between neutral-positive - > previous indicated sentiment score of 7-8 seems like a fit.


--------------------

Delivery time extraction

In [ ]:
answer=get_openai_response('question',dialog=organized_dialog)
print("Answer: ",answer)

Answer:  The appointment time for delivery is at 8 a.m. on Wednesday.


We've extracted the emotional identifiers from the text only , it would be interesting to see how these extracted emotions correlate with emotions extracted/identified from audio itself.Here we will use some pre-trained model , that probably won't give us a very accurate/indicative results. However ,as a start point, that can be later fine-tuned on our data , and used in combination with text classifier , it might be an interesting option.

In [16]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
#!pip install huggingsound

In [91]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
#!pip install git+https://github.com/huggingface/datasets.git
#!pip install git+https://github.com/huggingface/transformers.git
#!pip install torchaudio
#!pip install librosa

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from   transformers import AutoConfig, Wav2Vec2FeatureExtractor
import librosa
import IPython.display as ipd
import numpy as np
import pandas as pd

In [31]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss


@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)

class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model_name_or_path = "harshit345/xlsr-wav2vec-speech-emotion-recognition"
model_name_or_path = "ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition"
config = AutoConfig.from_pretrained(model_name_or_path)
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path)
sampling_rate = feature_extractor.sampling_rate
model = Wav2Vec2ForSpeechClassification.from_pretrained(model_name_or_path).to(device)

Some weights of Wav2Vec2ForSpeechClassification were not initialized from the model checkpoint at ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
import numpy as np
import soundfile as sf

we will chunk the audio file, make separate audio file for each conversational turn , and will classify them separately.

In [29]:
y, sr = librosa.load(audio_file, sr=16000, duration=2027)
all_segments=[]
for item in organized_segments:
    start = np.floor(item[0] * sr).astype(int)
    end = np.ceil(item[1] * sr).astype(int)
    all_segments.append(y[start:end])

In [ ]:
for i, chunk in enumerate(all_segments):
    output_file = f"/content/gdrive/MyDrive/gdrive/GP_Transco/chunk_{i}.wav"
    sf.write(output_file, chunk, sr)

In [38]:
def speech_file_to_array_fn(path, sampling_rate):
    speech_array, _sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech
def predict(path, sampling_rate):
    speech = speech_file_to_array_fn(path, sampling_rate)
    inputs = feature_extractor(speech, sampling_rate=sampling_rate, return_tensors="pt", padding=True)
    inputs = {key: inputs[key].to(device) for key in inputs}
    with torch.no_grad():
        logits = model(**inputs).logits
    scores = F.softmax(logits, dim=1).detach().cpu().numpy()[0]
    outputs = [{"Emotion": config.id2label[i], "Score": f"{round(score * 100, 3):.1f}%"} for i, score in enumerate(scores)]
    return outputs

In [39]:
all_scores=[]
for i in range(0,len(all_segments)):
    path = f"/content/gdrive/MyDrive/gdrive/GP_Transco/chunk_{i}.wav"
    all_scores.append(predict(path, sampling_rate))

In [40]:
ems=[]
for item in all_scores:
    dct={}
    for i in item:
        dct[i['Emotion']]=i['Score']
    ems.append(dct)

In [41]:
emotions_df=pd.DataFrame.from_records(ems)

In [42]:
emotions_df['speaker']=[list(item.keys())[0] for item in organized_dialog]
emotions_df['phrase']=[list(item.values())[0] for item in organized_dialog]
emotions_df['start_end_time']=organized_segments

here is the final dataframe, with spectrum of emotions , identified speaker, spoken phrase and time stamp.

In [44]:
emotions_df

,angry,calm,disgust,fearful,happy,neutral,sad,surprised,speaker,phrase,start_end_time
0,16.1%,9.0%,12.0%,12.3%,12.5%,10.0%,13.5%,14.6%,Truck Driver,"Hey Fidel, I'm sorry, this one.","(2.591, 4.472)"
1,16.9%,9.0%,16.2%,11.6%,11.5%,9.9%,12.6%,12.3%,Routing Operator,I'm done with the truck.,"(4.673, 6.194)"
2,16.8%,9.0%,13.1%,12.0%,12.0%,9.5%,13.2%,14.3%,Truck Driver,"Okay, perfect, perfect.","(6.854, 9.116)"
3,16.5%,9.0%,14.4%,11.2%,12.1%,9.8%,12.9%,14.2%,Routing Operator,What time is my clock going to kick in tomorrow?,"(9.216, 11.017)"
4,17.4%,8.8%,13.4%,12.5%,11.6%,10.0%,12.5%,13.8%,Truck Driver,Let me see real quick.One second.For tomorrow ...,"(11.938, 54.124)"
5,17.1%,8.6%,13.7%,12.2%,12.0%,10.2%,12.5%,13.6%,Routing Operator,"All right.Yeah, I'm not leaving until tomorrow...","(55.045, 58.987)"
6,15.1%,9.8%,13.2%,12.5%,11.9%,10.7%,13.8%,12.9%,Truck Driver,"No, no, it's fine, it's fine.Are you at HQ, r...","(59.611, 62.933)"
7,14.7%,9.7%,11.7%,12.0%,13.2%,10.4%,12.7%,15.6%,Routing Operator,"Yeah, I'm out of time.","(63.514, 64.934)"
8,16.0%,9.6%,13.6%,11.5%,12.0%,9.7%,13.3%,14.4%,Truck Driver,"No, it's fine, you can sit down at the headqua...","(65.515, 69.197)"
9,16.6%,8.4%,12.9%,12.6%,12.5%,10.4%,12.5%,14.3%,Routing Operator,They rule with YARMUV?They use YARMUV?,"(69.457, 73.34)"


 - Though many entries seem to have a quite negative emotional level , we know the model was pre-trained on recordings from actors performing on 8 different emotions , which is different from our target domain.
 - It would be desirainteresting to collect labeled samples of our data and to fine-tune the model. It definitely could be complementary to the textual emotion classifier.

In [45]:
emotions_df.to_csv('/content/gdrive/MyDrive/gdrive/GP_Transco/emotions_df.csv',index=False)